In [1]:
import inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
os.sys.path.insert(0,parentdir) 

import Artha.neo
from Artha.neo import Neo
import sqlite3
from tqdm import tqdm
import csv


def dict_factory(cursor, row):
    d = {}
    for idx, col in enumerate(cursor.description):
        d[col[0]] = str(row[idx])
    return d

neo = Neo("neo4j://localhost:7687", "neo4j", "part2")
conn = sqlite3.connect("../data/ucheckra_.db")
conn.row_factory = dict_factory
checkra_follows = conn.cursor().execute("select * from following").fetchall()
neo.session.run("CREATE CONSTRAINT twitter_id IF NOT EXISTS ON (n:Person) ASSERT n.id IS UNIQUE")
print("done")

done


In [2]:
print(len(neo.get_nodes()), len(neo.get_relations()))

58821 76261


In [ ]:
for record in neo.get_nodes():
    print(record["username"])

In [6]:
neo.clear_db()

AttributeError: 'Neo4jDriver' object has no attribute 'clear_db'

In [67]:
import json
path = "/Users/harshasomisetty/Library/Application Support/com.Neo4j.Relate/Data/dbmss/dbms-8c0b6b9b-8405-42f7-aee7-4bff9bf1d898/import/follows.csv"
def user_edges(username, location = "../data/users"):
    conn_temp = sqlite3.connect(location+f"/u{username}.db")
    conn_temp.row_factory = dict_factory


    try:
        temp_follows = conn_temp.cursor().execute("select * from following").fetchall()
        print(temp_follows[0])
    except:
        return -1
    else:

        with open(path, "w+") as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames = ["id", "name", "username"])
            writer.writeheader()
            writer.writerows(temp_follows)
            if username == "checkra_":
                neo.session.run(
                        "Merge (n:Person {username: 'checkra_'})"
                        "ON CREATE SET n.id ='1356259499431129092', n.name = 'Checkra', n.username = 'checkra_'")
            
            node_result = neo.session.run(
                        "LOAD CSV WITH HEADERS FROM 'file:/follows.csv' AS row "
                        "MERGE (n:Person {id:row.id}) "
                        "ON CREATE SET n.id = row.id, n.name=row.name, n.username=row.username "
                        "return n"
                        # "WITH n "
                        # # "MERGE (m:Person {username: '"+username+"'}) "
                        # "MERGE (:Person {username: '"+username+"'})-[r:FOLLOWS]->(n) "
                        # "return n, r"
                        )

            edge_result = neo.session.run(
                        "LOAD CSV WITH HEADERS FROM 'file:/follows.csv' AS row "
                        # "MATCH (n:Person {id: row.id}) "
                        # "WITH n "
                        "MERGE (m:Person {username: '"+username+"'}) "
                        # "MERGE (m)-[r:FOLLOWS]->(n) "
                        "return m"
                        )
            


        return node_result, edge_result

nodes, edges = user_edges("checkra_", location = "../data")
# nodes = user_edges("Ninjascalp")
node_records = [record for record in nodes]
edge_records = [record for record in edges]
print("done", len(node_records), len(edge_records))
print(len(neo.get_nodes()), len(neo.get_relations()))

{'id': '943958987556507653', 'name': 'Ninja', 'username': 'Ninjascalp'}
done 195 314
196 0


In [69]:
with open("text.txt", "w+") as w:
    for ind, record in enumerate(node_records):
        w.write(str(ind)+" "+str(record["n"]["name"])+"\n")

In [7]:
with open("text1.txt", "w+") as w:
    for ind, record in enumerate(edge_records):
        w.write(str(ind)+" "+str(record["n"]["name"])+" "+str(record["m"]["name"])+"\n")

In [124]:
with open("text1.txt", "w+") as w:
    for ind, record in enumerate(edge_records):
        w.write(str(ind)+" "+str(record["r"].nodes[1]["name"])+"\n")

In [123]:
for i in edge_records:
    print(i["r"].nodes[1]["name"])

Ninja
☣surferjim is more stoked than you- regularly.
Edward Morra
Andrew Wilkinson
AnthonΞ Sassano | sassal.eth 🦇🔊
Jim Scalpert (aka Elton Jim)
Crystal Lee 李佳佳
𝕮𝖍𝖎
Psycho™
Little Peepo
Alex
MultifractelFarol
Christopher Bloomstran
Jeremy Arnold
NFT MAXIMALIST 🧦
Theo📚
Murfski (KAVA Maximalist)
⚡️WAVE JUMPER⚡️
Ether Schtroumpf
Money Fox (EL LEGENDADO (of a new hand))
ChartingOptions
Daniel
Market Timer
Moonshine
$DOGE TO $1 🚀
emini Jingle 🛎🛎
SpaceX
Lex Fridman
Jeff Bezos
Bill Gates
Bob Kendall
Fermat's Library
David Sands
Marc Andreessen
Joe Rogan
Dr Jordan B Peterson
Vergil Den
jason@calacanis.com
Machine Learning Tweet Feed
ye
Bret Weinstein
Milto Savvidis
L/L Research
Ben Orlin
f i l ₿ f i l ₿
Yates Investing
George Friedman
Allen Institute for AI
Louie Gohmert
Ted Yoho
Nicholas Sandmann
AntonKreil
Sasha Rush
Grant Sanderson
Sean McFate
Kurzgesagt
CGP Grey 🐝
Chip Huyen
Senator Rand Paul
Tulsi Gabbard 🌺
Christina Qi
Chris Herd
Zack Kanter
Geopolitical Futures
Preethi Kasireddy
Amit Kum

In [112]:
for ind, record in enumerate(edge_records):
    # print(dir(record[""]))
    print(str(ind)+" "+edge_records["r"].nodes[1]["name"])

TypeError: list indices must be integers or slices, not str

In [ ]:
conn_temp = sqlite3.connect("../data/users/uBrianRoemmele.db")
conn_temp.row_factory = dict_factory


temp_follows = conn_temp.cursor().execute("select id, name, username from following").fetchall()

    # with open(path, "w+") as csvfile:
    #     writer = csv.DictWriter(csvfile, fieldnames = ["id", "name", "username", "by"])
    #     writer.writeheader()
    #     writer.writerows(temp_follows)

In [ ]:
%%time
temp_follows = conn_temp.cursor().execute("select id, name, username from following").fetchall()


    

In [ ]:
%%time

num = user_edges("Ninjascalp")
print("done", "Ninjascalp", num)
# for user in tqdm(checkra_follows):
#     num = user_edges(user["username"])
#     print("done", user["username"], num)

In [ ]:
checkra_follows[:15]

In [ ]:
import os
os.listdir()

In [ ]:
os.listdir("/Users/harshasomisetty/")

In [ ]:
os.getcwd()

In [10]:


from neo4j import GraphDatabase

neo = GraphDatabase.driver("neo4j://localhost:7687", auth = ("neo4j", "part2"))
session = neo.session()

def user_edges(username, location = "../data/users"):


        node_result = session.run(
                    "LOAD CSV WITH HEADERS FROM 'file:/follows.csv' AS row "
                    "MERGE (n:Person {id:row.id}) "
                    "ON CREATE SET n.id = row.id, n.name=row.name, n.username=row.username "
                    # "return n"
                    "WITH n "
                    "MERGE (m:Person {username: '"+username+"'}) "
                    "MERGE (:Person {username: '"+username+"'})-[r:FOLLOWS]->(n) "
                    "return n, m"
                    )

        # edge_result = session.run(
        #             "LOAD CSV WITH HEADERS FROM 'file:/follows.csv' AS row "
        #             # "MATCH (n:Person {id: row.id}) "
        #             # "WITH n "
        #             "MERGE (m:Person {username: '"+username+"'}) "
        #             # "MERGE (m)-[r:FOLLOWS]->(n) "
        #             "return m"
        #             )
            


        return node_result, edge_result

nodes, edges = user_edges("checkra_", location = "../data")
# nodes = user_edges("Ninjascalp")
node_records = [record for record in nodes]
edge_records = [record for record in edges]
print("done", len(node_records), len(edge_records))

done 314 98910


In [8]:
session.run("MATCH (n) DETACH DELETE (n)")